In [2]:
import pymatgen
import numpy as np
from mp_api.client import MPRester
import re
import pandas as pd



In [3]:
# Yoink all binary oxides from the materials project database. Find oxides with the lowest formation energy.
api_key = 'Lv3H4D1twE2kDiXZLWdaBTIRM82iRdCi'
mpr = MPRester(api_key)

c:\cleen\Apps\Anaconda\lib\site-packages\mp_api\client\mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [4]:
expstableoxides = mpr.summary.search(elements=["O"], theoretical=False, is_stable=True, fields=["material_id","formula_pretty","decomposes_to","formation_energy_per_atom","energy_per_atom"])
stableoxides = mpr.summary.search(elements=["O"], is_stable=True, fields=["material_id","formula_pretty","decomposes_to","formation_energy_per_atom","energy_per_atom"])

Retrieving SummaryDoc documents:   0%|          | 0/7477 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/10928 [00:00<?, ?it/s]

In [7]:
oxides = stableoxides
oxides_id = [oxide.material_id for oxide in oxides]
oxides_formula = [oxide.formula_pretty for oxide in oxides]
oxides_decompose = [oxide.decomposes_to for oxide in oxides]
oxides_formationenergyperatom = [oxide.formation_energy_per_atom for oxide in oxides]
oxides_energyperatom = [oxide.energy_per_atom for oxide in oxides]



In [8]:
oxides_data = np.column_stack((oxides_id, oxides_formula, oxides_formationenergyperatom, oxides_energyperatom))

oxides_data = np.savetxt('stableoxides_data.csv', oxides_data, delimiter=',', fmt='%s')

In [2]:
# Define a list of common oxides (simple combinations of elements and oxygen)
# Process data
oxidedata = np.genfromtxt('metaloxides.csv', delimiter=',', dtype=None, encoding=None)
inorg_oxides = oxidedata[:,0]
inorg_oxides_meltingpoint = oxidedata[:,1]
inorg_oxidespre = np.column_stack((inorg_oxides, inorg_oxides_meltingpoint))
inorg_oxidespre_bad = inorg_oxidespre[:,1] == 'NA'
inorg_oxides_init = inorg_oxidespre[~inorg_oxidespre_bad]
# inorg_oxides1 = inorg_oxidespre_1[:,0]
# inorg_oxides1_meltingpoints = inorg_oxidespre_1[:,1]

alloys = inorg_oxides_init[:,0]
# Regular expression pattern to match the chemical formulas
pattern = r'([A-Z][a-z]?)(\d*)'

# Lists to store the parsed data
elements = []
counts = []

# Iterate over each alloy name
for alloy in alloys:
    # Use regular expression to extract elements and counts
    matches = re.findall(pattern, alloy)
    
    # Extract the elements and counts from the matches
    alloy_elements = [match[0] for match in matches]
    alloy_counts = [int(match[1]) if match[1] else 1 for match in matches]
    
    # Append the elements and counts to the lists
    elements.append(alloy_elements)
    counts.append(alloy_counts)

# Convert the lists to numpy arrays
elements = np.array(elements)
counts = np.array(counts)
combined = np.column_stack((inorg_oxides_init[:,1], alloys, elements[:,0], counts[:,0], elements[:,1], counts[:,1]))


In [12]:
### from https://github.com/Bowserinator/Periodic-Table-JSON CITE this
periodictable = pd.read_csv('PeriodicTableCSV.csv')
symbols = periodictable[['symbol']]
elements = periodictable[['symbol','number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling']]
periodictable_digits = periodictable[['number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling']]
periodictable_ionizationE = periodictable[['ionization_energies']]
elements = elements.to_numpy()
periodictable_digits = np.nan_to_num(periodictable_digits.to_numpy(), nan=0)


## List form ionization E is wierd.
# Use first ionization energy for now
periodictable_ionizationE = np.nan_to_num(periodictable_ionizationE.to_numpy(), nan=0)
firstionizationE = [1]
for i in range(len(periodictable_ionizationE)):
    firstE = np.array(eval(periodictable_ionizationE[i][0]))
    if len(firstE) == 0:
        firstE = [500]
    firstionizationE = np.vstack((firstionizationE,firstE[0]))

firstionizationE = firstionizationE[1:len(firstionizationE)]
properties1 = ['symbol','number','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
properties2 = ['symbol','number','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']

periodictable_nums = np.column_stack((symbols,periodictable_digits, firstionizationE))
## Make each row have the properties listed. As an example for Li2O,
## Li2O Li 1 phase ionization_energies atomic_mass boil density melt electron_affinity electronegativity_pauling

# for index in range(len(combined)):
#     element1 = combined[index][2]
#     element2 = combined[index][4]
#     multiplier1 = combined[index][3]
#     multiplier2 = combined[index][5]
#     for elementindex in range(len(elements)):
#         if elements[elementindex][0] == element1:
#             element1nums = periodictable_nums[elementindex]*int(multiplier1)
            
#             ## element1_ionE = np.fromstring(periodictable_ionizationE[elementindex][0][1:-1], sep=',')*int(multiplier1)
#             properties1 = np.vstack((properties1,element1nums))

#         if elements[elementindex][0] == element2:
#             element2nums = periodictable_nums[elementindex]*int(multiplier2)
#             ## element2_ionE = np.fromstring(periodictable_ionizationE[elementindex][0][1:-1], sep=',')*int(multiplier2)
#             properties2 = np.vstack((properties2,element2nums))

In [13]:
PeriodicTable_Cleaned = ['symbol','number','period','group','Gas','Solid','Liquid','atomic_mass','boil','density','melt','electron_affinity','electronegativity_pauling','firstionE']
PeriodicTable_Cleaned = np.vstack((PeriodicTable_Cleaned, periodictable_nums))
np.savetxt('PeriodicTableCleaned_data.csv', PeriodicTable_Cleaned, delimiter=',',fmt='%s')


In [4]:
### Now to put it all together...
### Training set is the first 103 data points (Simple oxides)
### Past 103, are more complicated oxides. Can we predict them?
### Below are classifiers:
# 
combineddata = np.vstack((['Melting temp','Formula','Element1','#El1','Element2','#El2'], combined))
metaloxides_data = np.column_stack((combineddata[:,(0,1,2,3)],properties1,combineddata[:,(4,5)],properties2))
np.savetxt('metaloxides_data.csv', metaloxides_data, delimiter=',',fmt='%s')


In [8]:
############ MODEL TIME
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.nn.functional import normalize

In [9]:
metaloxides_data_shuffled = np.random.permutation(metaloxides_data[1:len(metaloxides_data)])
modelinputdata = metaloxides_data_shuffled[:,(0,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21)].astype(np.float32)
nptraining_set = modelinputdata[0:80][:,(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18)]
nptraining_target = modelinputdata[0:80][:,0]
npvalidation_set = modelinputdata[81:len(modelinputdata)][:,(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18)]
npvalidation_target =  modelinputdata[81:len(modelinputdata)][:,0]



In [35]:
training_set = torch.from_numpy(nptraining_set)
training_target = torch.from_numpy(nptraining_target)
validation_set = torch.from_numpy(npvalidation_set)
validation_target = torch.from_numpy(npvalidation_target)


# Define the neural network model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=18, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=1, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 64)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

# Create an instance of the model
model = CNNModel()

# Define the loss function and optimizer
criterion = nn.MSELoss()                           # Mean Squared Error loss
optimizer = optim.SGD(model.parameters(), lr=0.01) # Stochastic Gradient Descent optimizer with learning rate 0.01


In [36]:
# Perform the training loop
num_epochs = 50000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(training_set)
    loss = criterion(outputs, training_target)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    # After backward pass and before optimizer step
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()


    # Print the loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [80, 18]

In [31]:
# Evaluate the model on the testing set
with torch.no_grad():
    test_outputs = model(validation_set)
    test_loss = criterion(test_outputs, validation_target)
    print(f"Test Loss: {test_loss.item():.4f}")


Test Loss: 1064400.5000


c:\cleen\Apps\Anaconda\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [33]:

with torch.no_grad():
    predicted_output = model(validation_set)
    print(predicted_output)


tensor([[ 419.1455],
        [ 693.1042],
        [2030.8698],
        [1407.2300],
        [ 976.8240],
        [1529.8424],
        [1241.8934],
        [ 651.6829],
        [1548.6248],
        [ 375.2786],
        [1814.2365],
        [ 777.4400],
        [ 734.7039],
        [1763.6951],
        [1670.6499],
        [ 588.5369],
        [1175.9532],
        [1981.5344],
        [1237.8440],
        [1524.2148],
        [1802.8040]])


In [34]:
validation_target

tensor([ 118.4000, 1310.0000,  820.0000, 2400.0000, 2710.0000,  350.0000,
         360.0000,  795.0000,  290.0000,  300.0000,  450.0000,  -35.9000,
         733.0000, 1436.0000,  195.0000,  130.0000,  100.0000,  300.0000,
         862.0000, 1132.0000, 2300.0000])

In [4]:
# Initialize the Materials Project API key
api_key = 'Lv3H4D1twE2kDiXZLWdaBTIRM82iRdCi' 
# Initialize the Materials Project API
mpr = MPRester(api_key)


c:\cleen\Apps\Anaconda\lib\site-packages\mp_api\client\mprester.py:182: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [5]:
mpr.get_entry_by_material_id(inorg_oxides_init[1,0])

Retrieving ThermoDoc documents:   0%|          | 0/9 [00:00<?, ?it/s]

[mp-841-GGA ComputedStructureEntry - Li4 O4       (Li2O2)
 Energy (Uncorrected)     = -38.7689  eV (-4.8461  eV/atom)
 Correction               = -1.8600   eV (-0.2325  eV/atom)
 Energy (Final)           = -40.6289  eV (-5.0786  eV/atom)
 Energy Adjustments:
   MP2020 anion correction (peroxide): -1.8600   eV (-0.2325  eV/atom)
 Parameters:
   potcar_spec            = [{'titel': 'PAW_PBE Li_sv 23Jan2001', 'hash': '4799bab014a83a07c654d7196c8ecfa9'}, {'titel': 'PAW_PBE O 08Apr2002', 'hash': '7a25bc5b9a5393f46600a4939d357982'}]
   is_hubbard             = False
   hubbards               = {}
   run_type               = GGA
 Data:
   oxide_type             = peroxide
   aspherical             = True
   last_updated           = 2020-05-03 10:33:29.375000
   task_id                = mp-1422354
   material_id            = mp-841
   oxidation_states       = {'Li': 1.0, 'O': -1.0}
   run_type               = GGA,
 mp-1097030-GGA ComputedStructureEntry - Li1 O1       (Li2O2)
 Energy (Uncorrecte

In [9]:
test = mpr.summary.search(material_ids=["mp-841-GGA"])
list_of_available_fields = mpr.summary.available_fields


Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

In [10]:
list_of_available_fields = mpr.summary.available_fields


In [11]:
list_of_available_fields


['builder_meta',
 'nsites',
 'elements',
 'nelements',
 'composition',
 'composition_reduced',
 'formula_pretty',
 'formula_anonymous',
 'chemsys',
 'volume',
 'density',
 'density_atomic',
 'symmetry',
 'property_name',
 'material_id',
 'deprecated',
 'deprecation_reasons',
 'last_updated',
 'origins',
 'warnings',
 'structure',
 'task_ids',
 'uncorrected_energy_per_atom',
 'energy_per_atom',
 'formation_energy_per_atom',
 'energy_above_hull',
 'is_stable',
 'equilibrium_reaction_energy_per_atom',
 'decomposes_to',
 'xas',
 'grain_boundaries',
 'band_gap',
 'cbm',
 'vbm',
 'efermi',
 'is_gap_direct',
 'is_metal',
 'es_source_calc_id',
 'bandstructure',
 'dos',
 'dos_energy_up',
 'dos_energy_down',
 'is_magnetic',
 'ordering',
 'total_magnetization',
 'total_magnetization_normalized_vol',
 'total_magnetization_normalized_formula_units',
 'num_magnetic_sites',
 'num_unique_magnetic_sites',
 'types_of_magnetic_species',
 'k_voigt',
 'k_reuss',
 'k_vrh',
 'g_voigt',
 'g_reuss',
 'g_vrh',


In [8]:
query = f"melting point of NiO"
url = next(search(query, num_results=1))

        # Make a request to the search result URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the melting point information from the page
result = soup.find('div', class_='yxAsKe kZ91ed')
result

In [12]:
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Nickel(II) oxide - Wikipedia</title>
<script>document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.cookie.match(/(?:^|; )enwikimwclient